# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Get the data and save it to a variable.

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

Check the length of the column containing doi's.

In [3]:
len(CORD19_CSV['doi'])

77448

Display the column doi to see if there are inconsistencies such as NaN's

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

Create a series with solely unique values and neglect NaN's. It is important to sort the unique values. Otherwise, the method is creating different results after each restart of the notebook. 

In [5]:
doi_counted = doi.value_counts().sort_index(ascending=True)
doi_counted

10.1001/jamainternmed.2020.1369       1
10.1001/jamanetworkopen.2020.16382    1
10.1001/jamanetworkopen.2020.17521    1
10.1001/jamanetworkopen.2020.20485    1
10.1001/jamanetworkopen.2020.24984    1
                                     ..
10.9745/ghsp-d-20-00115               1
10.9745/ghsp-d-20-00171               1
10.9745/ghsp-d-20-00218               1
10.9758/cpn.2020.18.4.607             1
10.9781/ijimai.2020.02.002            1
Name: doi, Length: 74302, dtype: int64

The following function determines the requested information from the Scopus API. (https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4)

In [6]:
#Adapted from https://github.com/ElsevierDev/elsapy/blob/master/exampleProg.py
def fetch_scopus_api(client, doi):
    """obtain additional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

Thusly, the configuration file is set up and contains an APIkey. Further information: https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md

In [7]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

Moreover, the client is initialized with the API-Key.

In [8]:
client = ElsClient(config['apikey'])

For demonstation purposes, the following cells shows which data is returned by the Scopus API. 

In [9]:
return_example = fetch_scopus_api(client, '10.1016/j.dsx.2020.04.012')
print(json.dumps(return_example, indent=2))

{
  "affiliation": [
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Hamdard",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Millia Islamia",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Indraprastha Apollo Hospitals",
      "affiliation-country": "India"
    }
  ],
  "coredata": {
    "srctype": "j",
    "eid": "2-s2.0-85083171050",
    "pubmed-id": "32305024",
    "prism:coverDate": "2020-07-01",
    "prism:aggregationType": "Journal",
    "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85083171050",
    "dc:creator": {
      "author": [
        {
          "ce:given-name": "Raju",
          "preferred-name": {
            "ce:given-name": "Raju",
            "ce:initials": "R.",
            "ce:surname": "Vaishya",
            "ce:indexed-name": "Vaishya R."
          },
          "@seq": "1",
          "ce:init

Based on the returned data, further analysis is conductable. Therefore, two notebooks are created to analyse data linked to: 
<ul>
  <li>affiliation</li>
  <li>coredata</li>
</ul>    

Thusly, the already fetched coredata and affiliation are read and combined to a DataFrame for further processing.

In [10]:
df_current_extra_info = pd.DataFrame()
try:
    read_affiliation = pd.read_pickle('extra_info_affiliation_CS.pkl')
    read_coredata = pd.read_pickle('extra_info_coredata_CS.pkl')
    df_current_extra_info['affiliation'] = read_affiliation
    df_current_extra_info['coredata'] = read_coredata
    df_current_extra_info
except:
    print("The DataFrame is empty")
    #if the dataframe is not empty set the variable to show the dataframe

The length of the DataFrame containing the current information is assigned to a variable to be used for further processing. 
Therefore, the length will be used within a while loop as a starting index. 

In [11]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

46129

In [12]:
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': 'Palo Alto', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083266658', ..."
1,"[{'affiliation-city': 'Seattle', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
2,"[{'affiliation-city': 'Cambridge', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '8',..."
3,"[{'affiliation-city': 'Madison', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '9',..."
4,"[{'affiliation-city': 'Los Angeles', 'affilnam...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
...,...,...
46124,None,None
46125,None,None
46126,None,None
46127,None,None


In [13]:
def contains_only_None(dic):
    """
    This functions inspects an dictionary and returns True if it solely contains None values
    """
    return len(dic) == sum(value == None for value in dict_new_extra_info.values())

In [14]:
def append_fetched_data_to_df(df_current_extra_info, dic):
    """
    This function appends or inserts newly fetched data to the DataFrame containing scopus data.
    """
    #df_current_extra_info -> holding the latest data, new data needs to be appended to it, 
    #df_newly_fetched_transposed -> holdy newly fetched data, needs to be inserted or fetched
    
    if contains_only_None(dic):
        placeholder_entries = pd.DataFrame(np.empty((len(dict_new_extra_info),2),dtype=object),columns=['affiliation','coredata'], index=dict_new_extra_info.keys())
        df_newly_fetched_transposed = placeholder_entries
        print(placeholder_entries)
    else:
        #Prior appending, the dictionary is transformed to a DataFrame
        df_newly_fetched = pd.DataFrame(dic)
        #For readability, the DataFrame is transposed
        df_newly_fetched_transposed = df_newly_fetched.T
        print(df_newly_fetched_transposed)
    
    #Insert newly fetched rows which were previously not successful appended
    for index, row in df_newly_fetched_transposed.iterrows():
        #insert to current extra info DataFrame because the row is existent
        if index in df_current_extra_info.index and row.affiliation is not None:
            df_current_extra_info.loc[index] = row
        #append to current extra info DataFrame because the row is new     
        if index not in df_current_extra_info.index:
            df_current_extra_info = df_current_extra_info.append(row, ignore_index=True)
            
    #returning DataFrame with inserted and replaced rows. 
    return df_current_extra_info

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [15]:
def store_df_columns(df):
    ser_affiliation = df['affiliation']
    ser_coredata = df['coredata']
    ser_affiliation.to_pickle('extra_info_affiliation_CS.pkl')
    ser_coredata.to_pickle('extra_info_coredata_CS.pkl')
    return ser_affiliation, ser_coredata

In [16]:
# placeholder_entries = pd.DataFrame(np.empty((4,2),dtype=object),columns=['affiliation','coredata'])

In [17]:
# placeholder_entries

Subsequently, the fetched scopus data is stored within a dictionary. Besides, the print function is used to show the state of the process by displaying the latest fetched information. 

In [ ]:
%%time
dict_new_extra_info = dict()
len_dois = len(doi_counted)
def trigger_fetching():
    threshold = 0 
    i = len_df_current_extra_info
    while i < len_dois: #-> upto modified, normally len_dois
        dict_new_extra_info[i] = fetch_scopus_api(client, doi_counted.index[i])
        print("Position fetched: " + str(i) + " -> " +  doi_counted.index[i])
        i = i + 1 
        threshold = threshold + 1
        if threshold > 99:
            df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
            stored_series = store_df_columns(df_combined_extra_info)
            threshold = 0
            print("batch saved")
trigger_fetching()

Position fetched: 46129 -> 10.1101/2020.09.27.20202754
Position fetched: 46130 -> 10.1101/2020.09.27.315762
Position fetched: 46131 -> 10.1101/2020.09.27.316018
Position fetched: 46132 -> 10.1101/2020.09.27.316158
Position fetched: 46133 -> 10.1101/2020.09.28.20164798
Position fetched: 46134 -> 10.1101/2020.09.28.20190009
Position fetched: 46135 -> 10.1101/2020.09.28.20201475
Position fetched: 46136 -> 10.1101/2020.09.28.20201947
Position fetched: 46137 -> 10.1101/2020.09.28.20202028
Position fetched: 46138 -> 10.1101/2020.09.28.20202804
Position fetched: 46139 -> 10.1101/2020.09.28.20202911
Position fetched: 46140 -> 10.1101/2020.09.28.20202929
Position fetched: 46141 -> 10.1101/2020.09.28.20202937
Position fetched: 46142 -> 10.1101/2020.09.28.20202945
Position fetched: 46143 -> 10.1101/2020.09.28.20202952
Position fetched: 46144 -> 10.1101/2020.09.28.20203109
Position fetched: 46145 -> 10.1101/2020.09.28.20203166
Position fetched: 46146 -> 10.1101/2020.09.28.20203174
Position fetched

Position fetched: 46273 -> 10.1101/2020.10.05.20207001
Position fetched: 46274 -> 10.1101/2020.10.05.20207118
Position fetched: 46275 -> 10.1101/2020.10.05.20207217
Position fetched: 46276 -> 10.1101/2020.10.05.20207423
Position fetched: 46277 -> 10.1101/2020.10.05.325290
Position fetched: 46278 -> 10.1101/2020.10.05.326850
Position fetched: 46279 -> 10.1101/2020.10.06.20204487
Position fetched: 46280 -> 10.1101/2020.10.06.20205864
Position fetched: 46281 -> 10.1101/2020.10.06.20207571
Position fetched: 46282 -> 10.1101/2020.10.06.20207761
Position fetched: 46283 -> 10.1101/2020.10.06.20208033
Position fetched: 46284 -> 10.1101/2020.10.06.20208132
Position fetched: 46285 -> 10.1101/2020.10.06.323634
Position fetched: 46286 -> 10.1101/2020.10.06.327080
Position fetched: 46287 -> 10.1101/2020.10.06.327445
Position fetched: 46288 -> 10.1101/2020.10.06.327452
Position fetched: 46289 -> 10.1101/2020.10.06.327742
Position fetched: 46290 -> 10.1101/2020.10.06.328112
Position fetched: 46291 ->

Position fetched: 46418 -> 10.1101/2020.10.13.20211425
Position fetched: 46419 -> 10.1101/2020.10.13.20211664
Position fetched: 46420 -> 10.1101/2020.10.13.20211763
Position fetched: 46421 -> 10.1101/2020.10.13.20211771
Position fetched: 46422 -> 10.1101/2020.10.13.20211821
Position fetched: 46423 -> 10.1101/2020.10.13.20211854
Position fetched: 46424 -> 10.1101/2020.10.13.20211888
Position fetched: 46425 -> 10.1101/2020.10.13.20211912
Position fetched: 46426 -> 10.1101/2020.10.13.20211953
Position fetched: 46427 -> 10.1101/2020.10.13.20212035
Position fetched: 46428 -> 10.1101/2020.10.13.20212092
      affiliation coredata
46129        None     None
46130        None     None
46131        None     None
46132        None     None
46133        None     None
...           ...      ...
46424        None     None
46425        None     None
46426        None     None
46427        None     None
46428        None     None

[300 rows x 2 columns]
batch saved
Position fetched: 46429 -> 10.1101/

Position fetched: 46556 -> 10.1101/2020.10.19.346031
Position fetched: 46557 -> 10.1101/2020.10.20.20210195
Position fetched: 46558 -> 10.1101/2020.10.20.20213116
Position fetched: 46559 -> 10.1101/2020.10.20.20213793
Position fetched: 46560 -> 10.1101/2020.10.20.20215541
Position fetched: 46561 -> 10.1101/2020.10.20.20215608
Position fetched: 46562 -> 10.1101/2020.10.20.20215616
Position fetched: 46563 -> 10.1101/2020.10.20.20215715
Position fetched: 46564 -> 10.1101/2020.10.20.20215756
Position fetched: 46565 -> 10.1101/2020.10.20.20215814
Position fetched: 46566 -> 10.1101/2020.10.20.20215863
Position fetched: 46567 -> 10.1101/2020.10.20.20215905
Position fetched: 46568 -> 10.1101/2020.10.20.20215962
Position fetched: 46569 -> 10.1101/2020.10.20.20215970
Position fetched: 46570 -> 10.1101/2020.10.20.20216150
Position fetched: 46571 -> 10.1101/2020.10.20.20216283
Position fetched: 46572 -> 10.1101/2020.10.20.20216291
Position fetched: 46573 -> 10.1101/2020.10.20.20216309
Position fet

Position fetched: 46700 -> 10.1101/2020.10.27.20211631
Position fetched: 46701 -> 10.1101/2020.10.27.20215566
Position fetched: 46702 -> 10.1101/2020.10.27.20216366
Position fetched: 46703 -> 10.1101/2020.10.27.20219196
Position fetched: 46704 -> 10.1101/2020.10.27.20219717
Position fetched: 46705 -> 10.1101/2020.10.27.20220061
Position fetched: 46706 -> 10.1101/2020.10.27.20220400
Position fetched: 46707 -> 10.1101/2020.10.27.20220442
Position fetched: 46708 -> 10.1101/2020.10.27.20220541
Position fetched: 46709 -> 10.1101/2020.10.27.20220640
Position fetched: 46710 -> 10.1101/2020.10.27.20220665
Position fetched: 46711 -> 10.1101/2020.10.27.20220715
Position fetched: 46712 -> 10.1101/2020.10.27.20220723
Position fetched: 46713 -> 10.1101/2020.10.27.20220830
Position fetched: 46714 -> 10.1101/2020.10.27.20220863
Position fetched: 46715 -> 10.1101/2020.10.27.20220897
Position fetched: 46716 -> 10.1101/2020.10.27.20220905
Position fetched: 46717 -> 10.1101/2020.10.27.354563
Position fet

...

Position fetched: 53863 -> 10.1177/0020764020966020
Position fetched: 53864 -> 10.1177/0020764020966021
Position fetched: 53865 -> 10.1177/0020764020970996
Position fetched: 53866 -> 10.1177/0020764020971318
Position fetched: 53867 -> 10.1177/0020764020972445
Position fetched: 53868 -> 10.1177/0020764020979025
Position fetched: 53869 -> 10.1177/0020764020979732
Position fetched: 53870 -> 10.1177/0020872820941748
Position fetched: 53871 -> 10.1177/0020872820959706
Position fetched: 53872 -> 10.1177/0022034520946251
Position fetched: 53873 -> 10.1177/0022034520962087
Position fetched: 53874 -> 10.1177/0022034520970536
Position fetched: 53875 -> 10.1177/0022034520979644
Position fetched: 53876 -> 10.1177/0022167820927807
Position fetched: 53877 -> 10.1177/0022167820934226
Position fetched: 53878 -> 10.1177/0022167820959488
Position fetched: 53879 -> 10.1177/0022429420967008
Position fetched: 53880 -> 10.1177/0030222820966928
Position fetched: 53881 -> 10.1177/0030727020939592
Position fet

Position fetched: 54021 -> 10.1177/1040638720910107
Position fetched: 54022 -> 10.1177/1048291120932913
Position fetched: 54023 -> 10.1177/1048291120974358
Position fetched: 54024 -> 10.1177/1049732320912409
Position fetched: 54025 -> 10.1177/1049732320951145
Position fetched: 54026 -> 10.1177/1049732320951526
Position fetched: 54027 -> 10.1177/1049732320952314
Position fetched: 54028 -> 10.1177/1049909120942494
batch saved
Position fetched: 54029 -> 10.1177/1049909120943577
Position fetched: 54030 -> 10.1177/1049909120948235
Position fetched: 54031 -> 10.1177/1054773820927144
Position fetched: 54032 -> 10.1177/1060028019846118
Position fetched: 54033 -> 10.1177/1062860620933587
Position fetched: 54034 -> 10.1177/1062860620946741
Position fetched: 54035 -> 10.1177/1062860620949141
Position fetched: 54036 -> 10.1177/1066480720969194
Position fetched: 54037 -> 10.1177/1073110520917000
Position fetched: 54038 -> 10.1177/1073274820960467
Position fetched: 54039 -> 10.1177/1073858420956905


Position fetched: 54179 -> 10.1177/1745506520969616
Position fetched: 54180 -> 10.1177/1745691620942516
Position fetched: 54181 -> 10.1177/1745691621989243
Position fetched: 54182 -> 10.1177/1747016120931998
Position fetched: 54183 -> 10.1177/1747493020937397
Position fetched: 54184 -> 10.1177/1747493020938301
Position fetched: 54185 -> 10.1177/1747493020959216
Position fetched: 54186 -> 10.1177/1747493020968236
Position fetched: 54187 -> 10.1177/1750458920978858
Position fetched: 54188 -> 10.1177/1751143720937957
Position fetched: 54189 -> 10.1177/1751143720961672
Position fetched: 54190 -> 10.1177/1751143720969587
Position fetched: 54191 -> 10.1177/1753425920971347
Position fetched: 54192 -> 10.1177/1753466620933510
Position fetched: 54193 -> 10.1177/1753466620937175
Position fetched: 54194 -> 10.1177/1753466620963017
Position fetched: 54195 -> 10.1177/1753466620963026
Position fetched: 54196 -> 10.1177/1753466620963035
Position fetched: 54197 -> 10.1177/1753466620971146
Position fet

Position fetched: 54336 -> 10.1177/2150132720980612
Position fetched: 54337 -> 10.1177/2150132720981297
Position fetched: 54338 -> 10.1177/2150132720981540
Position fetched: 54339 -> 10.1177/2150132720985055
Position fetched: 54340 -> 10.1177/2150132720987715
Position fetched: 54341 -> 10.1177/2150135120938330
Position fetched: 54342 -> 10.1177/2150135120950256
Position fetched: 54343 -> 10.1177/2150135120954816
Position fetched: 54344 -> 10.1177/2151459320951721
Position fetched: 54345 -> 10.1177/2151459320969377
Position fetched: 54346 -> 10.1177/2151459320972674
Position fetched: 54347 -> 10.1177/2151459320979974
Position fetched: 54348 -> 10.1177/2152656720928245
Position fetched: 54349 -> 10.1177/2158244015611711
Position fetched: 54350 -> 10.1177/2158244019827717
Position fetched: 54351 -> 10.1177/2164956120973622
Position fetched: 54352 -> 10.1177/2164956120973636
Position fetched: 54353 -> 10.1177/2164956120975369
Position fetched: 54354 -> 10.1177/2164956120976554
Position fet

Position fetched: 54492 -> 10.1183/13993003.01704-2020
Position fetched: 54493 -> 10.1183/13993003.01727-2020
Position fetched: 54494 -> 10.1183/13993003.01759-2020
Position fetched: 54495 -> 10.1183/13993003.01809-2020
Position fetched: 54496 -> 10.1183/13993003.01811-2020
Position fetched: 54497 -> 10.1183/13993003.01826-2019
Position fetched: 54498 -> 10.1183/13993003.01912-2020
Position fetched: 54499 -> 10.1183/13993003.01935-2020
Position fetched: 54500 -> 10.1183/13993003.01998-2020
Position fetched: 54501 -> 10.1183/13993003.02060-2020
Position fetched: 54502 -> 10.1183/13993003.02113-2020
Position fetched: 54503 -> 10.1183/13993003.02144-2020
Position fetched: 54504 -> 10.1183/13993003.02197-2020
Position fetched: 54505 -> 10.1183/13993003.02226-2020
Position fetched: 54506 -> 10.1183/13993003.02281-2020
Position fetched: 54507 -> 10.1183/13993003.02378-2020
Position fetched: 54508 -> 10.1183/13993003.02384-2020
Position fetched: 54509 -> 10.1183/13993003.02429-2020
Position f

The following cell is useful when the process above is interrupted. Therefore, the dictionary containing fetched information can be narrowed down to useful entries. 

In [ ]:
# def save_new_extra_info(len_df_current_extra_info, upto):
#     """
#     This function is used to separate successfull API calls from API calls which were prevented due to an invalid API-Key.
#     As a result, this function returns a range of valid entries up to the given parameter. 
#     """
#     dict_new_extra_info_saver = dict()
#     i = len_df_current_extra_info
#     while i < upto:
#         #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
#         dict_new_extra_info_saver[i] = dict_new_extra_info[i]
#         i = i + 1 
#     return dict_new_extra_info_saver

The existing and newly fetched information are combined into one DataFrame. 

In [ ]:
# df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
df_combined_extra_info

In [ ]:
#to big for GitHub
#df_combined_extra_info.to_csv('extra_info_CS5099.csv', sep='\t')

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [ ]:
# stored_series = store_df_columns(df_combined_extra_info)
# stored_series[0]

In [ ]:
# stored_series[1]

Verifying that the returned None values are due to non existent data and not to an invalid API-Key

In [ ]:
# len_data = len(stored_series[0])
# len_data 

In [ ]:
# ser_doi = pd.Series(doi_counted.index[:len_data])
# ser_doi

In [ ]:
# df_current_extra_info_checker = df_combined_extra_info
# df_current_extra_info_checker['doi'] = ser_doi

In [ ]:
# %%time
# len_df_current_extra_info_checker = len(df_current_extra_info_checker)
# dict_new_extra_info_checker = dict()
# i = 0 
# while i < len_df_current_extra_info_checker: ###################################################### 
#     if df_current_extra_info_checker['affiliation'][i] == None:
#         dict_new_extra_info_checker[i] = fetch_scopus_api(client, ser_doi[i])
#         print("Position fetched again: " + str(i) + " -> " +  ser_doi[i])
#     i = i + 1    

In [ ]:
# dict_new_extra_info_checker
# -> check if at least one value is not None -> otherwise the process is finished here

In [ ]:
# len(dict_new_extra_info_checker)

In [ ]:
# df_combined_extra_info_fetched_again  = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
# df_combined_extra_info_fetched_again

In [ ]:
# store_df_columns(df_combined_extra_info_fetched_again)

In [ ]:
# df_combined_extra_info_fetched_again['check_doi'] = ser_doi
# df_combined_extra_info_fetched_again.head(30)